# Jaccard Similarity

In this notebook, we will use cuGraph to compute the Jaccard similarity and run analytics on our training dataset

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Last Edit: 03/08/2019

RAPIDS Versions: 0.6.0    
    


## Introduction
The Jaccard similarity between two sets is defined as the ratio of the volume of their intersection divided by the volume of their union.  $Jaccard(A, B) = \frac{|(A \cap B)|}{|(A \cup B)|}$

In the context of graphs, the neighborhood of a vertex defines its set. The Jaccard similarity weight of each edge represents the strength of connection between vertices based on the relative similarity of their neighbors. For further detail see [Wikipedia](https://en.wikipedia.org/wiki/Jaccard_index)

To compute the Jaccard similarity between all-pairs of vertices connected by an edge in cuGraph use: 
**nvJaccard(G)**
* __G__: A cugraph.Graph object

Returns:
* __df__: cudf.DataFrame with three names columns:
    * df["source"]: The source vertex id.
    * df["destination"]: The destination vertex id.
    * df["jaccard_coeff"]: The jaccard coefficient computed between the source and destination vertex.
    
    
## cuGraph 0.6 Notice 
cuGraph version 0.6 has a number of limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations    
    


    

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)

This is a small graph which allows for easy visual inspection to validate results.  

In [1]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

In [2]:
# define a function for printing the top most similar vertices
def printMostSimilar(df):
    bestEdge = 0
    
    #find the best
    for i in range(len(df)):
        if df['jaccard_coeff'][i] > df['jaccard_coeff'][bestEdge]:
            bestEdge = i
        
    print("Vertices " + str(df['source'][bestEdge]) + " and " + 
          str(df['destination'][bestEdge]) + " are most similar with score: " 
          + str(df['jaccard_coeff'][bestEdge]))

In [3]:
# define a function for printing similar vertices based on a threshold
def print_gt_threshold(df, limit):
    
    for i in range(len(df)):
        if df['jaccard_coeff'][i] > limit:
            print("Vertices " + str(df['source'][i]) + " and " + 
                  str(df['destination'][i]) + " are similar with score: " + 
                  str(df['jaccard_coeff'][i]))

### Read the CSV datafile using cuDF

In [4]:
# Test file  
datafile='../data/networks/karate-data.csv'

# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

In [5]:
# Let's look at the DataFrame. There should be two columns and 154 records
gdf

<cudf.DataFrame ncols=2 nrows=154 >

In [6]:
# Look at the first few data records - the output should be two colums src and dst
gdf.head().to_pandas()

,src,dst
0,1,2
1,1,3
2,1,4
3,1,5
4,1,6


### Create a Graph and call the Jaccard analytic

In [7]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"])

In [8]:
# How many vertices are in the graph?  Remember that Graph is zero based while teh data start at vertex 1
G.num_vertices()

35

_The Jaccard test graph has only 34 vertices, so why is the Graph listing 35?_

As mentioned above, cuGraph vertex numbering is zero-based, meaning that the first vertex ID starts at zero.  The test dataset is 1-based.  Because of that, the Graph object adds an extra isolated vertex with an ID of zero.  Hence the difference in vertex count.  
The next version of cuGraph will add a function to address this issue.  

In [ ]:
# Call cugraph.nvJaccard 
df = cugraph.nvJaccard(G)

In [ ]:
printMostSimilar(df)

The Most similar shoul be 33 and 34.
Vertex 33 has 12 neighbors, vertex 34 has 16 neighbors.  They share 10 neighbors in common:
$jaccard = 10 / (10 + (12 -10) + (16-10)) = 10 / 18 = 0.55$

In [ ]:
print_gt_threshold(df, 0.4)

In [ ]:
# Since it is a small graph we can print all scores.
# Notice that only connected vertices are computed

# let's sort the data first
g = df.groupby(['jaccard_coeff'], method='cudf')


In [ ]:
df_s = g.as_df()

In [ ]:
# The groupby as_df function returns a tuple where the first item is the dataframe
print_gt_threshold(df_s[0], 0.0)

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___